In [1]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine

import getpass  

In [2]:
password = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

1. How many distinct (different) actors' last names are there?

In [4]:
distinct_actor_lastname = pd.read_sql_query("""SELECT count(distinct(last_name)) as 'Count' from actor;""",engine)
distinct_actor_lastname

,Count
0,121


2. Add an additional column day_type with values 'weekend' and 'workday' depending on the rental day of the week.

In [5]:
data_rental = pd.read_sql_query("""SELECT *, date_format(rental_date,'%a') as 'Day',
CASE WHEN 'Day' in ('Sat','Sun') THEN 'Weekend'
     ELSE 'Weekday'
END AS 'day_type'
FROM rental;""",engine)

data_rental = pd.DataFrame(data_rental,columns=data_rental.columns)
data_rental = data_rental.drop(['Day'],axis=1)
display(data_rental.head())

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,day_type
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,Weekday
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,Weekday
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,Weekday
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,Weekday
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,Weekday


3. Get all films with ARMAGEDDON in the title.

In [16]:
data_film_name_alike=pd.read_sql_query('''SELECT title from film
WHERE title LIKE "%ARMAGEDDON%";''',engine)
data_film_name_alike

ValueError: unsupported format character 'A' (0x41) at index 42

4. Get 10 the longest films.

In [43]:
result_long_film = engine.execute("""SELECT title, length from film 
ORDER BY length DESC 
LIMIT 10;""")

for row in result_long_film:
    print(row)

('DARN FORRESTER', 185)
('POND SEATTLE', 185)
('CHICAGO NORTH', 185)
('MUSCLE BRIGHT', 185)
('WORST BANGER', 185)
('GANGS PRIDE', 185)
('SOLDIERS EVOLUTION', 185)
('HOME PITY', 185)
('SWEET BROTHERHOOD', 185)
('CONTROL ANTHEM', 185)


In [44]:
result_long_film1 = pd.read_sql_query("""SELECT title, length from film 
ORDER BY length DESC
LIMIT 10;""",engine)
result_long_film1

,title,length
0,DARN FORRESTER,185
1,POND SEATTLE,185
2,CHICAGO NORTH,185
3,MUSCLE BRIGHT,185
4,WORST BANGER,185
5,GANGS PRIDE,185
6,SOLDIERS EVOLUTION,185
7,HOME PITY,185
8,SWEET BROTHERHOOD,185
9,CONTROL ANTHEM,185


5. How many films include Behind the Scenes content?

In [47]:
result_bts = pd.read_sql_query("""SELECT count(title) FROM film
WHERE special_features LIKE 'Behind the Scenes';""",engine)
result_bts

,count(title)
0,70


6. Which kind of movies (rating) have a mean duration of more than two hours?

In [48]:
result_duration_rating = pd.read_sql_query("""SELECT rating, avg(length) as 'Mean Duration' FROM film
GROUP BY rating
ORDER BY avg(length) DESC;""",engine)
result_duration_rating.head()

,rating,Mean Duration
0,PG-13,120.4439
1,R,118.6615
2,NC-17,113.2286
3,PG,112.0052
4,G,111.0506


7. Rank films by length (filter out the rows that have nulls or 0s in length column). In your output, only select the columns title, length, and the rank.

In [50]:
result_film_rank = pd.read_sql_query("""SELECT title, length, dense_rank () over (ORDER BY length DESC) FROM film
WHERE length is not null AND length > 0;""",engine)
result_film_rank.head(50)

,title,length,dense_rank () over (ORDER BY length DESC)
0,CHICAGO NORTH,185,1
1,CONTROL ANTHEM,185,1
2,DARN FORRESTER,185,1
3,GANGS PRIDE,185,1
4,HOME PITY,185,1
5,MUSCLE BRIGHT,185,1
6,POND SEATTLE,185,1
7,SOLDIERS EVOLUTION,185,1
8,SWEET BROTHERHOOD,185,1
9,WORST BANGER,185,1
